# Flowers Recognition

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
import os
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam

In [ ]:
X = []
y = []
flower_classes = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [ ]:
def prepare_data(flower_class):
    flower_dir = "flowers/{}".format(flower_class)
    for img in tqdm(os.listdir(flower_dir)):
        path = os.path.join(flower_dir, img)
        if not img.startswith('.') and os.path.isfile(path):
            label = flower_class

            img = cv2.imread(path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (150, 150))

            X.append(np.array(img))
            y.append(str(label))


In [ ]:
for f in flower_classes: 
    prepare_data(f)

### Visualization

In [ ]:
sns.countplot(y)

### Preprocess

In [ ]:
X = np.array(X)

label = LabelEncoder()
y = label.fit_transform(y)
y = to_categorical(y, 5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [ ]:
training_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False)

training_generator = training_datagen.flow(X_train, y_train, batch_size=32) 

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow(X_test, y_test, batch_size=32)

### Training

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5,5), padding='Same', activation='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Conv2D(96, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Conv2D(96, (3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(
    training_generator,
    steps_per_epoch=X_train.shape[0]//32,
    validation_data=validation_generator, 
    validation_steps=X_test.shape[0]//32,
    epochs=50)

### Evaluation

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')

In [ ]:
'''
Inspiration
1. https://www.kaggle.com/rajmehra03/flower-recognition-cnn-keras
'''